In [1]:
--Exercise 8.1

--Red-Black Tree
data Color = R | B deriving (Show)
data Tree a = 
    E |
    T {
        _color :: Color,
        _left :: Tree a,
        _value :: (Bool, a),
        _right :: Tree a
    } deriving (Show)
    
data SizedTree t a = ST (Int, Int) (t a) deriving (Show)

In [2]:
--Functions for Tree
memberT :: Ord a => a -> Tree a -> Bool
memberT x E = False
memberT x (T _ lt (ind, y) rt) 
    | x < y = memberT x lt
    | x > y = memberT x rt
    | otherwise = ind
    
insertT :: Ord a => a -> Tree a -> Tree a
insertT x t = let
    ins E = T R E (True, x) E
    ins t@(T c lt node@(ind, y) rt) 
        | x < y = balance c (ins lt) node rt
        | x > y = balance c lt node (ins rt)
        | otherwise = t
    T _ lt y rt = ins t
    in
    T B lt y rt
    
balance :: Color -> Tree a -> (Bool, a) -> Tree a -> Tree a
balance B (T R (T R a x b) y c) z d = T R (T B a x b) y (T B c z d)
balance B (T R a x (T R b y c)) z d = T R (T B a x b) y (T B c z d)
balance B a x (T R (T R b y c) z d) = T R (T B a x b) y (T B c z d)
balance B a x (T R b y (T R c z d)) = T R (T B a x b) y (T B c z d)
balance c a x b = T c a x b

deleteT :: Ord a => a -> Tree a -> Tree a
deleteT x E = E
deleteT x t@(T c lt node@(ind, y) rt) 
    | x < y = T c (deleteT x lt) node rt
    | y < x = T c lt node (deleteT x rt)
    | otherwise = T c lt (False, y) rt 

toOrdList :: Ord a => Tree a -> [a]
toOrdList E = []
toOrdList t@(T _ lt (ind, x) rt) = if ind == True
    then toOrdList lt ++ [x] ++ toOrdList rt
    else toOrdList lt ++ toOrdList rt

data Digit a = One a (Tree a) | Two a (Tree a) a (Tree a) deriving (Show)

fromOrdList :: Ord a => [a] -> Tree a
fromOrdList = foldl link E . foldr add [] 

link :: Ord a => Tree a -> Digit a -> Tree a
link treeSoFar (One a t) = T B treeSoFar (True, a) t
link treeSoFar (Two a1 t1 a2 t2) = T B (T R treeSoFar (True, a1) t1) (True, a2) t2 

add :: Ord a => a -> [Digit a] -> [Digit a]
add a [] = [One a E]
add a ((One a1 t1) : ds) = Two a E a1 t1 : ds
add a ((Two a1 t1 a2 t2) : ds) = One a E : (incr (One a1 (T B t1 (True, a2) t2) : ds))
    where
        incr [] = []
        incr [d] = [d]
        incr ((One a1 t1) : (One a2 t2) : ds) = Two a1 t1 a2 t2 : ds
        incr ((One a1 t1) : (Two a2 t2 a3 t3) : ds) = (One a1 t1) : (One a2 (T B t2 (True, a3) t3)) : incr ds

Line 36: Redundant ==
Found:
ind == True
Why not:
indLine 51: Redundant bracket
Found:
(One a1 t1) : ds
Why not:
One a1 t1 : dsLine 52: Redundant bracket
Found:
(Two a1 t1 a2 t2) : ds
Why not:
Two a1 t1 a2 t2 : dsLine 52: Redundant bracket
Found:
One a E : (incr (One a1 (T B t1 (True, a2) t2) : ds))
Why not:
One a E : incr (One a1 (T B t1 (True, a2) t2) : ds)Line 56: Redundant bracket
Found:
(One a1 t1) : ((One a2 t2) : ds)
Why not:
One a1 t1 : ((One a2 t2) : ds)Line 56: Redundant bracket
Found:
(One a2 t2) : ds
Why not:
One a2 t2 : dsLine 57: Redundant bracket
Found:
(One a1 t1) : ((Two a2 t2 a3 t3) : ds)
Why not:
One a1 t1 : ((Two a2 t2 a3 t3) : ds)Line 57: Redundant bracket
Found:
(Two a2 t2 a3 t3) : ds
Why not:
Two a2 t2 a3 t3 : dsLine 57: Redundant bracket
Found:
(One a1 t1) : (One a2 (T B t2 (True, a3) t3)) : incr ds
Why not:
One a1 t1 : (One a2 (T B t2 (True, a3) t3)) : incr dsLine 57: Redundant bracket
Found:
(One a2 (T B t2 (True, a3) t3)) : incr ds
Why not:
One a2 (T B t2 (True, a3) t3) : incr ds

In [3]:
--Functions for SizedTree
empty = ST (0, 0) E

member :: Ord a => a -> SizedTree Tree a -> Bool
member x (ST _ t) = memberT x t

insert :: Ord a => a -> SizedTree Tree a -> SizedTree Tree a
insert x (ST (dSize, fSize) t) = ST (dSize, fSize+1) (insertT x t)

delete :: Ord a => a -> SizedTree Tree a -> SizedTree Tree a
delete x (ST (dSize, fSize) t) = if (dSize+1) > (fSize `div` 2)
    then rebuild (deleteT x t)
    else ST (dSize+1, fSize) (deleteT x t)
    
rebuild :: Ord a => Tree a -> SizedTree Tree a
rebuild t = ST (0, length ordLst) (fromOrdList ordLst)
    where ordLst = toOrdList t 

In [4]:
--Test case from https://github.com/rst76
main = do
  print $ flip (foldr delete) "ab" $ foldr insert empty "ebdac"
  -- => T [2,5] B (T [] B E (False,'a') E) (False,'b') (T [] R (T [] B E (True,'c') E) (True,'d') (T [] B E (True,'e') E))
  print $ flip (foldr delete) "abc" $ foldr insert empty "ebdac"
  -- => T [0,2] B E (True,'d') (T [0,1] R E (True,'e') E)

main

ST (2,5) (T {_color = B, _left = T {_color = B, _left = E, _value = (False,'a'), _right = E}, _value = (False,'b'), _right = T {_color = R, _left = T {_color = B, _left = E, _value = (True,'c'), _right = E}, _value = (True,'d'), _right = T {_color = B, _left = E, _value = (True,'e'), _right = E}}})
ST (0,2) (T {_color = B, _left = T {_color = R, _left = E, _value = (True,'d'), _right = E}, _value = (True,'e'), _right = E})

In [2]:
:l Queue.hs

In [5]:
--Exercise 8.2 & 8.3

--HoodMelvilleQueue
module HoodMelvilleQueue (HoodMelvilleQueue) where

import Prelude hiding (head, tail)
import Queue

data RotationState a = 
       Idle
    | Reversing Int [a] [a] [a] [a]
    | Appending Int [a] [a]
    | Done [a] deriving (Show)

data HoodMelvilleQueue a = HM Int [a] (RotationState a) [a] deriving (Show)

exec diff (Reversing ok (x : f) f' (y : r) r') = (diff+2, Reversing (ok+1) f (x : f') r (y : r'))
exec diff (Reversing ok [] f' [y] r') = (diff+1, Appending ok f' (y : r'))
exec diff (Appending 0 f' r') = (diff, Done r')
exec diff (Appending ok (x : f') r') = (diff, Appending (ok-1) f' (x : r'))
exec diff state = (diff, state)

invalidate (Reversing ok f f' r r') = Reversing (ok-1) f f' r r'
invalidate (Appending 0 f' (x : r')) = Done r'
invalidate (Appending ok f' r') = Appending (ok-1) f' r'
invalidate state = state

exec1 diff f state r =
    case exec diff state of
        (trckedDiff, Done newf) -> HM trckedDiff newf Idle r
        (trckedDiff, newstate) -> HM trckedDiff f newstate r

exec2 diff f state r =
    case uncurry exec (exec diff state) of 
        (trckedDiff, Done newf) -> HM trckedDiff newf Idle r
        (trckedDiff, newstate) -> HM trckedDiff f newstate r

check diff f state r = 
    if 0 <= diff then exec1 diff f state r
    else let newstate = Reversing 0 f [] r []
            in exec2 0 f newstate []

instance Queue HoodMelvilleQueue where
    empty = HM 0 [] Idle []
    isEmpty (HM diff f state r) = null f

    snoc (HM diff f state r) x = check (diff-1) f state (x : r)

    head (HM _ [] _ _) = error "Error: Empty Queue"
    head (HM _ (x : f') _ _) = x

    tail (HM diff [] state r) = error "Error: Empty Queue"
    tail (HM diff (x : f') state r) = 
        check (diff-1) f' (invalidate state) r  

In [7]:
--Test case from https://github.com/rst76

import HoodMelvilleQueue

main :: IO ()
main = do
  let qs = scanl snoc empty [1 .. 7] :: [HoodMelvilleQueue Int]
  mapM_ print qs
  -- => HM [] 0 Idle []
  -- => HM [1] 1 Idle []
  -- => HM [1] 0 Idle [2]
  -- => HM [1] 3 (Appending 1 [1] [2,3]) []
  -- => HM [1] 2 (Appending 0 [] [1,2,3]) [4]
  -- => HM [1,2,3] 1 Idle [5,4]
  -- => HM [1,2,3] 0 Idle [6,5,4]
  -- => HM [1,2,3] 4 (Reversing 2 [3] [2,1] [5,4] [6,7]) []
  mapM_ print $ take 8 $ iterate tail $ last qs
  -- => HM [1,2,3] 4 (Reversing 2 [3] [2,1] [5,4] [6,7]) []
  -- => HM [2,3] 5 (Reversing 2 [] [3,2,1] [4] [5,6,7]) []
  -- => HM [3] 5 (Appending 1 [3,2,1] [4,5,6,7]) []
  -- => HM [4,5,6,7] 4 Idle []
  -- => HM [5,6,7] 3 Idle []
  -- => HM [6,7] 2 Idle []
  -- => HM [7] 1 Idle []
  -- => HM [] 0 Idle []

main

HM 0 [] Idle []
HM 1 [1] Idle []
HM 0 [1] Idle [2]
HM 3 [1] (Appending 1 [1] [2,3]) []
HM 2 [1] (Appending 0 [] [1,2,3]) [4]
HM 1 [1,2,3] Idle [5,4]
HM 0 [1,2,3] Idle [6,5,4]
HM 4 [1,2,3] (Reversing 2 [3] [2,1] [5,4] [6,7]) []
HM 4 [1,2,3] (Reversing 2 [3] [2,1] [5,4] [6,7]) []
HM 5 [2,3] (Reversing 2 [] [3,2,1] [4] [5,6,7]) []
HM 5 [3] (Appending 1 [3,2,1] [4,5,6,7]) []
HM 4 [4,5,6,7] Idle []
HM 3 [5,6,7] Idle []
HM 2 [6,7] Idle []
HM 1 [7] Idle []
HM 0 [] Idle []

In [1]:
:l Queue.hs

In [5]:
--Deque
module OutputRestrictedDeque where

import Prelude hiding(head, tail)

class OutputRestrictedDeque q where
    empty :: q a
    isEmpty :: q a -> Bool
    
    cons :: a -> q a -> q a
    head :: q a -> a
    tail :: q a -> q a
    
    snoc :: q a -> a -> q a
    --last :: q a -> a
    --init :: q a -> q a

In [6]:
:l Deque.hs
:l HoodMelvilleQueue.hs

In [14]:
--Exercise 8.4
import OutputRestrictedDeque
import qualified Queue as Q 
import HoodMelvilleQueue

data ListedQueue q a = LQ [a] (q a)

instance Q.Queue q => OutputRestrictedDeque (ListedQueue q) where
    empty = LQ [] Q.empty
    isEmpty (LQ lst q) = null lst && Q.isEmpty q
    
    cons x (LQ lst q) = LQ (x : lst) q
    head empty = error "Error: Empty Deque"
    head (LQ [] q) = Q.head q
    head (LQ (x : xs) q) = x
    tail empty = error "Error: Empty Deque"
    tail (LQ [] q) = LQ [] (Q.tail q)
    tail (LQ (x : xs) q) = LQ xs q
    
    snoc (LQ lst q) x = LQ lst (Q.snoc q x)